In [ ]:
!pip install lmfit

In [ ]:
import json

import lmfit
import matplotlib.pyplot as plt
import pandas as pd
from fucciphase import logistic
from fucciphase.phase import estimate_cell_phase_from_max_intensity
from fucciphase.plot import plot_normalized_intensities, plot_raw_intensities
from fucciphase.sensor import FUCCISASensor
from fucciphase.utils import normalize_channels

## Example data

Example data was based on 100x videos of HaCaT cells with TEMPOphase sensor.

In [ ]:
df = pd.read_csv("example_data/example_data_tempophase.csv")

In [ ]:
df["TRACK_ID"] = 1
df["FRAME"] = df.index

# Normalize curves again

In [ ]:
normalize_channels(df, ["cyan", "magenta"])

In [ ]:
plot_raw_intensities(
    df,
    "cyan",
    "magenta",
    "tab:cyan",
    "m",
    time_column="percentage",
    time_label="Percentage w.r.t. total time",
)
plt.show()

In [ ]:
plot_normalized_intensities(
    df,
    "cyan",
    "magenta",
    "tab:cyan",
    "m",
    time_column="percentage",
    time_label="Percentage w.r.t. total time",
)
plt.show()

In [ ]:
thresholds = [0.1, 0.1]
fuccisa = FUCCISASensor(
    phase_percentages=[0.0, 50.0, 50.0],
    center=[0.0, 0.0, 0.0, 0.0],
    sigma=[0.0, 0.0, 0.0, 0.0],
)

In [ ]:
estimate_cell_phase_from_max_intensity(
    df, ["cyan", "magenta"], fuccisa, background=[0.0, 0.0], thresholds=[0.1, 0.1]
)

In [ ]:
g1 = df.loc[df["DISCRETE_PHASE_MAX"] == "G1", "percentage"]
g1_perc = g1.max()
g1s = df.loc[df["DISCRETE_PHASE_MAX"] == "G1/S", "percentage"]
# filter mislabel
g1s_perc = g1s[(g1s < 80)].max()
sg2m_perc = 100.0 - g1s_perc

In [ ]:
print(g1_perc)
print(g1s_perc - g1_perc)
print(sg2m_perc)

# Define fit model

In [ ]:
model = lmfit.models.RectangleModel(form="logistic")


def fit_curve(time, data):
    """Fit a curve to step model."""
    parameters = model.guess(data, x=time)
    parameters["amplitude"].value = 1.0
    parameters["amplitude"].vary = False

    parameters["center1"].min = 0
    parameters["center2"].min = 0
    if parameters["center1"].value > parameters["center2"].value:
        parameters["center1"].value = 10000
        parameters["center2"].value = 50000

    curve_shifted = data - data.min()
    fit = model.fit(curve_shifted / curve_shifted.max(), parameters, x=time)
    print(fit.fit_report())
    return fit

## Process the DataFrame

In [ ]:
center = [0] * 4
sigma = [0] * 4

In [ ]:
percentages = df["percentage"]
fit = fit_curve(percentages, df["cyan_NORM"])
c1 = fit.params["center1"].value
c2 = fit.params["center2"].value
peak = fit.params["amplitude"].value
sigma1 = fit.params["sigma1"].value
sigma2 = fit.params["sigma2"].value
plt.plot(percentages, fit.best_fit, "o-", label="Best fit")
plt.plot(percentages, fit.data, label="Data")

plt.plot(
    percentages,
    1.0 - logistic(percentages, c1, sigma1),
    "o",
    label="Accumulation",
    markevery=3,
)
plt.plot(
    percentages,
    1.0 - logistic(percentages, c2, sigma2, sign=-1.0),
    "v",
    label="Degradation",
    markevery=3,
)

plt.vlines(g1_perc, 0, peak, color="black")
plt.vlines(g1s_perc, 0, peak, color="black", linestyles="dotted")

plt.ylabel("Normalised intensity")
plt.xlabel("Percentage w.r.t. total time")
plt.legend()
plt.savefig("cyan_fit.png")
plt.savefig("cyan_fit.svg")
plt.savefig("cyan_fit.pdf")
plt.show()

center[0] = c1
center[1] = c2
sigma[0] = sigma1
sigma[1] = sigma2

In [ ]:
percentages = df["percentage"]
fit = fit_curve(percentages, df["magenta_NORM"])
c1 = fit.params["center1"].value
c2 = fit.params["center2"].value
peak = fit.params["amplitude"].value
sigma1 = fit.params["sigma1"].value
sigma2 = fit.params["sigma2"].value
plt.plot(percentages, fit.best_fit, "o-", label="Best fit")
plt.plot(percentages, fit.data, label="Data")

plt.plot(
    percentages,
    1.0 - logistic(percentages, c1, sigma1),
    "o",
    label="Accumulation",
    markevery=3,
)
plt.plot(
    percentages,
    1.0 - logistic(percentages, c2, sigma2, sign=-1.0),
    "v",
    label="Degradation",
    markevery=3,
)

plt.vlines(g1_perc, 0, peak, color="black")
plt.vlines(g1s_perc, 0, peak, color="black", linestyles="dotted")

plt.ylabel("Normalised intensity")
plt.xlabel("Percentage w.r.t. total time")
plt.legend()
plt.savefig("magenta_fit.png")
plt.savefig("magenta_fit.svg")
plt.savefig("magenta_fit.pdf")
plt.show()

center[2] = c1
center[3] = c2
sigma[2] = sigma1
sigma[3] = sigma2

In [ ]:
result_dict = {
    "phase_percentages": [g1_perc, g1s_perc - g1_perc, sg2m_perc],
    "center": center,
    "sigma": sigma,
}

In [ ]:
with open("fuccisa_hacat.json", "w") as fp:
    json.dump(result_dict, fp)